# Predicting Movie Revenue

## Features  

**Variable to Predict:** Revenue


- Budget  
- Runtime  
- Genre  
- Release Date  
 - Season  
 - Holiday   
- Production Companies  
 - Number of companies involved  
 - Number of movies a company has made (previously)  
- Crew
 - Number of crewmembers  
- Cast
 - Gender Ratio for top 2/5/10/25
 - Number of previous movies top 10 cast have been in
- Collection
 - Is sequel (order)  


In [58]:
import numpy as np
import pandas as pd
import sqlite3
import holidays

pd.set_option('display.float_format','{:.4f}'.format)

In [3]:
db = sqlite3.connect(r'../../../Data Science Data/Unit 3/db.sqlite')

In [4]:
## SQL import statements to create dataframes

# Base dataframe to join other onto
base_df = pd.read_sql("""SELECT movie_id, budget, release_date, revenue, runtime, imdb_budget, imdb_revenue, title, collection 
                         FROM movies 
                         WHERE (budget > 0 OR imdb_budget > 0) AND (revenue > 0 OR imdb_revenue > 0)
                      """, db, index_col='movie_id', parse_dates=['release_date'])
base_df.index = base_df.index.map(int)

# Pull in genre table to create dummy variables
genre_df = pd.read_sql("""SELECT * FROM genres""", db)

# Create series for number of production companies associated with each movie_id
number_of_prod_companies = pd.read_sql("""SELECT movie_id, COUNT(company_name) AS prod_company_count
                                          FROM production_companies
                                          GROUP BY movie_id
                                          """, db, index_col='movie_id')
number_of_prod_companies.index = number_of_prod_companies.index.map(int)

production_companies = pd.read_sql("""
    SELECT *
    FROM production_companies
    """, db)
production_companies['movie_id'] = production_companies['movie_id'].map(int)

# Create series for number of crew members associated with each movie_id
number_of_crewmembers = pd.read_sql("""SELECT movie_id, COUNT(crew_member_id) AS crewmember_count
                                       FROM crew
                                       GROUP BY movie_id
                                       """, db, index_col='movie_id')
number_of_crewmembers.index = number_of_crewmembers.index.map(int)

# Dataframe for cast data to create gender ratio, and number of previous movies for cast
cast_data = pd.read_sql("""SELECT cc.movie_id, cc.cast_member_id, cc.cast_order, c.gender, m.release_date
                           FROM cast_credit cc
                           JOIN cast c ON cc.cast_member_id = c.cast_member_id
                           JOIN movies m ON cc.movie_id = m.movie_id
                                  """, db)
cast_data.index = cast_data.index.map(int)

In [5]:
# Create feature to note if the movie is a sequel
base_df.sort_values(['collection', 'release_date'], inplace=True)
base_df['order_in_collection'] = 1

current = None
counter = 1

for index, row in base_df.iterrows():
    if row['collection'] is None:
        break
    if current != row['collection']:
        counter = 1
    base_df.set_value(index,'order_in_collection', counter)
    current = row['collection']
    counter += 1

In [6]:
# Create feature for season
seasons = {
    1:'winter',
    2:'winter',
    3:'spring',
    4:'spring',
    5:'spring',
    6:'summer',
    7:'summer',
    8:'summer',
    9:'fall',
    10:'fall',
    11:'fall',
    12:'winter'
}

def season_for_date(date):
    if pd.notnull(date):
        return seasons[date.month]
    else:
        return None

movie_season = base_df['release_date'].apply(season_for_date)
movie_season.rename('season', inplace=True);

In [7]:
# Create feature for holiday
us_holidays = holidays.UnitedStates()

def check_holiday(date):
    if pd.notnull(date):
        return date in us_holidays
    else:
        return None
    
movie_is_holiday = base_df['release_date'].apply(check_holiday)
movie_is_holiday.rename('is_holiday', inplace=True);

In [8]:
# Create feature for production company experience
production_companies_joined = production_companies.merge(base_df, left_on='movie_id', right_index=True, how='left')

production_companies_joined.sort_values(['company_name', 'release_date'], inplace=True)
production_companies_joined['movies_produced'] = 0

current = None
counter = 0

for index, row in production_companies_joined.iterrows():
    if current != row['company_name']:
        counter = 0
    production_companies_joined.set_value(index,'movies_produced', counter)
    current = row['company_name']
    counter += 1
    
production_company_experience = production_companies_joined.groupby(by='movie_id').sum()['movies_produced']

In [9]:
# Create feature for cast experience
cast_data.sort_values(['cast_member_id', 'release_date'], inplace=True)
cast_data['movie_experience'] = 0

current = None
counter = 0

for index, row in cast_data.iterrows():
    if current != row['cast_member_id']:
        counter = 0
    cast_data.set_value(index,'movie_experience', counter)
    current = row['cast_member_id']
    counter += 1
    
cast_experience = cast_data[cast_data['cast_order'] <= 10].groupby('movie_id').sum()['movie_experience']

In [10]:
# Create dummy variables for genre
pivot_genre = genre_df.pivot_table(index='movie_id',columns='genre_name', values='genre_name',aggfunc=len, fill_value=0)
pivot_genre.index = pivot_genre.index.map(int)
pivot_genre = pivot_genre.apply(pd.to_numeric)

In [16]:
# Create column for gender ratio
gender_data = cast_data[cast_data['gender'] != 0]
gender_data = gender_data[gender_data['cast_order'] <= 5]
gender_data['gender'] = gender_data['gender'] - 1
gender_data = gender_data.groupby(by='movie_id').mean()['gender']
gender_data.index = gender_data.index.map(int)

In [42]:
# Join 'em up
df = base_df.join(pivot_genre, how='inner')
df = df.join(number_of_prod_companies, how='inner')
df = df.join(number_of_crewmembers, how='inner')
df = df.join(gender_data, how='inner')
df = df.join(cast_experience, how='inner')
df = df.join(production_company_experience, how='inner')
df = df.join(movie_is_holiday, how='inner')
df = df.join(movie_season, how='inner')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6195 entries, 117263 to 394117
Data columns (total 36 columns):
budget                 6195 non-null int64
release_date           6195 non-null datetime64[ns]
revenue                6195 non-null int64
runtime                6188 non-null float64
imdb_budget            1938 non-null object
imdb_revenue           2007 non-null object
title                  6195 non-null object
collection             1254 non-null object
order_in_collection    6195 non-null int64
Action                 6195 non-null int64
Adventure              6195 non-null int64
Animation              6195 non-null int64
Comedy                 6195 non-null int64
Crime                  6195 non-null int64
Documentary            6195 non-null int64
Drama                  6195 non-null int64
Family                 6195 non-null int64
Fantasy                6195 non-null int64
Foreign                6195 non-null int64
History                6195 non-null int64
Horror     

In [43]:
df['complete_budget'] = df['budget'].replace(0,np.nan).fillna(df['imdb_budget'])
df['complete_revenue'] = df['revenue'].replace(0,np.nan).fillna(df['imdb_revenue'])

df['complete_budget'] = pd.to_numeric(df['complete_budget'],errors='coerce')
df['complete_revenue'] = pd.to_numeric(df['complete_revenue'],errors='coerce')

In [44]:
df['is_sequel'] = df['order_in_collection'] > 1

In [45]:
cols_to_drop = ['imdb_budget','imdb_revenue','collection','order_in_collection']
df.drop(cols_to_drop,axis=1,inplace=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5687 entries, 117263 to 394117
Data columns (total 35 columns):
budget                5687 non-null int64
release_date          5687 non-null datetime64[ns]
revenue               5687 non-null int64
runtime               5687 non-null float64
title                 5687 non-null object
Action                5687 non-null int64
Adventure             5687 non-null int64
Animation             5687 non-null int64
Comedy                5687 non-null int64
Crime                 5687 non-null int64
Documentary           5687 non-null int64
Drama                 5687 non-null int64
Family                5687 non-null int64
Fantasy               5687 non-null int64
Foreign               5687 non-null int64
History               5687 non-null int64
Horror                5687 non-null int64
Music                 5687 non-null int64
Mystery               5687 non-null int64
Romance               5687 non-null int64
Science Fiction       5687 non-nul

In [62]:
df['complete_budget'].describe()

count        5687.0000
mean     28185572.7630
std      37388217.8597
min             1.0000
25%       5000000.0000
50%      15000000.0000
75%      35000000.0000
max     380000000.0000
Name: complete_budget, dtype: float64